In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! pip install pip install -U sentence-transformers

In [59]:
import pandas as pd
import numpy as np 
df = pd.read_csv('../input/phone-reviews/phone_reviews.csv')
df.head()

In [60]:
asin_rev_title_dict = {}
asin_review_dict = {}
title , body, stars= [] ,[] , []
for idx , lis in enumerate(df['review_title'].apply(eval)):
    title.extend(lis)
    asin_rev_title_dict[df.loc[idx,'asin']] = lis
for idx , lis in enumerate(df['review_body'].apply(eval)):
    body.extend(lis)
    asin_review_dict[df.loc[idx,'asin']] = lis
for lis in df['stars'].apply(eval):
    stars.extend(lis)
p_review = title + body

In [61]:
asin_list, title_list, body_list  , stars_list =  [] , [] , [], []
for idx , lis in enumerate(df['review_title'].apply(eval)):
    asin_list.extend([df.loc[idx,'asin']]*len(lis))
    title_list.extend(lis)
for idx , lis in enumerate(df['review_body'].apply(eval)):
    body_list.extend(lis)
for idx , lis in enumerate(df['stars'].apply(eval)):
    stars_list.extend(lis)

In [62]:
review_df_asins = pd.DataFrame({'asin': asin_list, 'title':title_list, 'body':body_list, 'stars':stars_list})
print(review_df_asins.shape)
review_df_asins.head()

In [63]:
review_df_asins['text'] = review_df_asins.title + review_df_asins.body  # concatenating both reviews and review body 
review_df_asins

In [64]:
r_df = review_df_asins.drop_duplicates('body').reset_index(drop = True)  # droping duplicates --review body from dataframe

title_body = list(r_df.title.values) +  list(r_df.body.values)  # concatenating both reviews and review body 

print(f'total reviews: {len(title_body)}\nrepeated_reviews: {len(title_body) - len(set(title_body))}')
p_review1 = list(set(title_body))
print('length of review list after removing duplicates:', len(p_review1))

In [65]:
r_df.loc[:,'text'] = r_df['title'] + r_df['body']

In [66]:
r_df = r_df.drop(['title','body','stars'], axis = 1)

In [67]:
from scipy import spatial

In [68]:
import re 

def preprocess(phrase):
    """ Given a phrase function 
    returns the processed phrase """
    
    phrase = re.sub(r'\n' , '.' , phrase)
    phrase = re.sub(r'\.\.+', ' .', phrase) # substitutes multiple fullstop to a single fullstop 
    phrase = phrase.lower()     # lower
    phrase = re.sub(r"the media could not be loaded", " ", phrase) # some reviews have this extra lines
    
    
    #decontractions 
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r" n ", " and ", phrase)
    phrase = re.sub(r" ?phn ?", " ", phrase)
    phrase = re.sub(r" ?phone ?", " ", phrase)
    phrase = re.sub(r" ?mobile ?", " ", phrase)
    phrase = re.sub(r" ?smartphone ?", " ", phrase)
    
    lis = list(filter(lambda x: x != ' ' or x != '', phrase.split('.')))

    final_to_encode = []
    for l in lis:
        final_to_encode.extend(list(filter(lambda x: x != ' ' or x != '', l.split(','))))
    
    final_to_encode2 = []    
    for l in final_to_encode:
        final_to_encode2.extend(list(filter(lambda x: x != ' ' or x != '', l.split(' and '))))
    
    final_to_encode3 = []
    for l in final_to_encode2:
        final_to_encode3.extend(list(filter(lambda x: x != ' ' or x != '', l.split(' but '))))

    return final_to_encode3
    

def aspect_sentiment(sentence_list , model ,aspects,  encoded_aspects , aux_sentences ):    
    results_tup = []
    
    encoded_list = model.encode(sentence_list)
    for i in range(len(encoded_list)):
        results = []
        for idx, aspect in enumerate(aspects):

            result = 1 - spatial.distance.cosine(encoded_aspects[idx], encoded_list[i])

            results.append(result)
        key = np.array(aspects)[np.argsort(results)[::-1]][0] 
        value = np.sort(results)[::-1][0]
        results_tup.append((key, value, sentence_list[i]))

    final_dict = {}
    for idx , r in enumerate(results_tup):
        aspect, cos_score ,sentence = r
        if len(sentence.strip().split()) > 1 and cos_score >= 0.35:
            compound_score = sid.polarity_scores(sentence)['compound']
            
            if aspect not in aux_sentences: 
                rat = 1 - spatial.distance.cosine(model.encode(f'{aspect} is good',  show_progress_bar  = False), 
                                                  model.encode(sentence,  show_progress_bar  = False) )
                rat2 = 1 - spatial.distance.cosine(model.encode(f'{aspect} is bad',  show_progress_bar  = False),
                                                   model.encode(sentence,  show_progress_bar  = False) )
                if rat>rat2:
                    sentiment = 'positive'

                else:
                    sentiment = 'negative'

            else:
                print('using sentences')
                rat = 1 - spatial.distance.cosine(model.encode(aux_sentences.get(aspect)[0] , show_progress_bar = False),
                                                  model.encode(sentence, show_progress_bar  = False) )
                rat2 = 1 - spatial.distance.cosine(model.encode(aux_sentences.get(aspect)[1] , show_progress_bar  = False), 
                                                   model.encode(sentence, show_progress_bar = False) )


                if rat>rat2:
                    sentiment = 'positive'
                else:
                    sentiment = 'negative'

            if aspect in final_dict:
                final_dict[aspect + "_" + str(idx)] = sentiment
            else:
                final_dict[aspect] = sentiment
    
    no_aspect = 'No aspects were found'
    return final_dict if final_dict != {} else no_aspect  

In [24]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [25]:
aspect_cluster = pd.read_csv('../input/final-p/final_p5.csv')
import pickle
with open('../input/arch-all/processed_reviews.pickle', 'rb') as f:
    rv4 = pickle.load(f)
with open('../input/arch-all/processed_reviews_emb_phase4.pickle', 'rb') as f:
    emb = pickle.load(f)    

In [69]:

import nltk
nltk.download('vader_lexicon')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

aspect_cluster = pd.read_csv('../input/final-p/final_p5.csv')
aspect_cluster2 = pd.read_csv('../input/df-top2/df_top (2).csv')

In [ ]:
# done project today, this is it , just seperate with comma as well ,and give aspects  #338 as example 
# remove jack  # 901 example not working #8695  # below 0.3 -- none if more than one seperated
# 2956 has and etc so does not work why not split with . and then , and then and and combine all the aspects
# 695 for error analysis , 18938 , 28958 , 3406

In [87]:
all_aspects = list(set(list(map(lambda x: x.strip() , ','.join(list(aspect_cluster['aspects'])).split(',')))))
all_aspects += list(set(list(map(lambda x: x.strip() , ','.join(list(aspect_cluster2['aspects'])).split(',')))))

all_aspects.extend(['recommendation', 'use', 'usage', 'built', 'body', 'color', 'design'])

In [88]:
words_to_remove = ['None', 'ad' , 'advertisments' , 'back', 'back cover' , 'battery drain', 'battery lif', 'calls', 'camera clarity'
 , 'camera megapixel' , 'charge',
'charge speed', 'dual bands',  'charge time',
'feature', 'feel', 'font' , 'frames per second', 'glass','graphic','hangs','haptic','hardware','jack','lags', 'lens', 'look' ,
'megapixel','memory', 'mic','none', 'overall' , 'picture clarity',  'power' , 'quality', 'reboot issue',
'regular usage', 'response', 'restart', 'sd slot','use', 'volumne','warp charger','wireless', 'worth' , 'phone built', 
                  'phone body', 'phone color', 'phone design', 'phone looks']

aa = list(set(all_aspects))

In [89]:
for word in words_to_remove:
    aa.remove(word)
all_aspects = aa

In [90]:
auxilary_sentences = {'sd card slot': ['there is a dedicated sd card slot' , 'no dedicated sd card slot'], 
                          'ads': ['there are no ads', 'there are ads'] , 
                         'heating': ['there is no heating issue' , 'there is heating issue'] ,
                         'glitches' : ['there are no glitches', 'there are glitches' ] , 
                          'call recording': ['there is call recording option','there is no call recording option'], 
                         'unwanted apps': ['there are no unwanted apps',  'there are unwanted apps'],
                         'hang': ['it does not hang', 'it does hang'] ,\
                          'lag': ['it does not lag', 'it does lag'] ,
                         'bgmi' : ['bgmi works well', 'bgmi does not work well'], 
                         'pubg' : ['pubg works well' , 'pubg does not works well'], 
                         'cod' : ['cod works well', 'cod does not works well'], 
                         'sd card slot': ['there is no sim card slot problem' , ' there is sim card slot problem '], 
                        'pre installed apps' : ['there are no pre installed apps',  'there are pre installed apps' ] ,
                         'screen flickering': ['there is no screen flickering' , 'there is screen flickering'] ,
                         'bloatware': [ 'there are no bloatware' , 'there are bloatwares'] , 
                         'dual bands': ['does support dual bands' , 'does not support dual bands' ],
                         'tint' : ['there is no tint', 'there is tint'],
                         'price' : ['price is low' , 'price is high'], 
                     'refund' : ['no refund needed' , 'asked for refund']}

In [91]:
import pickle 

with open('aux_sentences.pickle', 'wb') as f:
    pickle.dump(auxilary_sentences, f)

In [92]:
with open('aspects.pickle', 'wb') as f:
    pickle.dump(all_aspects, f)

In [93]:
aspect_idx_dict = dict(zip(all_aspects, range(len(all_aspects))))

In [94]:
encoded_aspects = model.encode(all_aspects)

In [95]:
p = 'phone looks amazing, camera is bad, good for regular use'

preprocessed = preprocess(p)
result = aspect_sentiment(preprocessed , model ,all_aspects,  encoded_aspects , auxilary_sentences )

In [96]:
result

In [97]:
def preprocess_and_predict(text, model, aspects, encoded_aspects , auxilary_sentences):
    preprocessed = preprocess(text)
    result = aspect_sentiment(preprocessed , model ,aspects,  encoded_aspects , auxilary_sentences )
    return result

In [100]:
text = 'the phone looks amazing, camera is the best '
aas= preprocess_and_predict(text, model, all_aspects, encoded_aspects , auxilary_sentences)
aas

In [79]:
review_df_asins['text']